# Import Packages & Load Data

In [ ]:
import glob
import os
import shutil
import json
import csv
import zipfile

In [ ]:
!pip install demoji
import demoji
demoji.download_codes()
import nltk
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()
import re
import shutil
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import string
import itertools
punctuation = string.punctuation
#Adding stop words
stopwordsset = set(stopwords.words("english"))
stopwordsset.add('rt')
stopwordsset.add("'s")
stopwordsset.add('...')
stopwordsset.add('...')
stopwordsset.add('"')
stopwordsset.add('na')
stopwordsset.add('re')
stopwordsset.add('u')
stopwordsset.add('x')
stopwordsset.add('@')
stopwordsset.add(':')
stopwordsset.add('#')
stopwordsset.add("'")
stopwordsset.add("’")
stopwordsset.add("‘")
stopwordsset.add('q')
stopwordsset.add('n')
stopwordsset.add('1')
stopwordsset.add('w')
stopwordsset.add('2')
stopwordsset.add('6')
stopwordsset.add('m')
stopwordsset.add('–')
stopwordsset.add('“')
stopwordsset.add('”')
stopwordsset.add('5')
stopwordsset.add('im')
stopwordsset.add("‘")
stopwordsset.add('amp')
stopwordsset.add('2021')
stopwordsset.add('…')
stopwordsset.add('210310')
stopwordsset.add('the...')
stopwordsset.add('the…')
stopwordsset.add('check')
stopwordsset.add('nothing')
stopwordsset.add('maybe')
stopwordsset.add('see')
stopwordsset.add('let')
stopwordsset.add('nayeon')
stopwordsset.add('one')
stopwordsset.add('first')
stopwordsset.add('come')
stopwordsset.add('soon')
stopwordsset.add('time')
stopwordsset.add('finally')
stopwordsset.add('think')
stopwordsset.add('please')

... OK (Got response in 0.16 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#connec to Google Drive to access fils
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
WORKING_DIR = "/content/drive/MyDrive/APRD6343/NetworkAnalysis"

In [ ]:
#Return the names of the zip files in the directory as a list
tweetzipfiles = glob.glob('%s/tweets/*.zip' % WORKING_DIR)

In [ ]:
tweetzipfiles

['/content/drive/MyDrive/APRD6343/NetworkAnalysis/tweets/balenciaga_lang:en.zip',
 '/content/drive/MyDrive/APRD6343/NetworkAnalysis/tweets/louis_vuitton_lang:en.zip',
 '/content/drive/MyDrive/APRD6343/NetworkAnalysis/tweets/gucci_lang:en.zip']

# Identify Unique Users in the Network

In [ ]:
# Identify unique users in the mention network
uniqueusers = {}

for tweetzipfile in tweetzipfiles: #Each file represents a zip file containing multiple tweets
  zf = zipfile.ZipFile(tweetzipfile) #Load zip file
  for i, obj in enumerate(zf.infolist()):
    tweetjson = json.load(zf.open(obj))
    userwhotweeted = tweetjson['user']['screen_name']  #Extract usernames from tweets
    followercount = tweetjson['user']['followers_count']
    if followercount > 15000: #Look at Twitter users with over 15k followers
      if userwhotweeted in uniqueusers:
        uniqueusers[userwhotweeted] += 1 
      if userwhotweeted not in uniqueusers: 
        uniqueusers[userwhotweeted] = 1

In [ ]:
len(uniqueusers)

366

In [ ]:
#Narrow down list of users to analyze
userstoinclude = set()
usercount = 0
for auser in uniqueusers:
    if uniqueusers[auser] != 0:
        usercount += 1
        userstoinclude.add(auser)
        
print(len(userstoinclude))

366


# Export Mentions Edgelist

In [ ]:
#Edgelist is a list of all connections beetween the Twitter users
edgelist = open('%s/designer15k.edgelist.for.gephi.csv' % WORKING_DIR, 'w')
csvwriter = csv.writer(edgelist)
header = ['Source', 'Target']
csvwriter.writerow(header)

15

In [ ]:
#Open one tweet file at a time, then load the json into the dictionary tweetjson

print('Writing edge list')
count = 0

for tweetzipfile in tweetzipfiles:
  count += 1
  if count % 1000 == 0:
    print(count)
  zf = zipfile.ZipFile(tweetzipfile)
  for i, obj in enumerate(zf.infolist()): #Enumerate each file in the zip file
    tweetjson = json.load(zf.open(obj)) 
    userwhotweeted = tweetjson['user']['screen_name'] #Pull out Twitter usernames
    if userwhotweeted in userstoinclude: 
#Within the tweet metadata, in the entities section, user_mentions field contains the entity that the current tweet mentions
      users = tweetjson['entities']['user_mentions'] #Pull who the tweet mentioned 
      if len(users) > 0:
          for auser in user
                screenname = auser['screen_name']
                row = [userwhotweeted, screenname] #List of users who sent out the tweet and person they mentioned in the tweet
                csvwriter.writerow(row)
edgelist.close()


Writing edge list


# Preprocess Tweets

In [ ]:
#Removing urls
def removeURL(text):
  result = re.sub(r"http\S+", "", text)
  return result

#Extracting contextual words from a sentence
#Tokenizing: taking out all the words in a sentence and turning it into tokens/words
def tokenize(text):
  #lower case
  text = text.lower()
  #split into individual words
  words = word_tokenize(text)
  return words

#Stem: reduce the number of repeated words, example - peaches : peach
def stem(tokenizedtext):
  rootwords = []
  for aword in tokenizedtext:
    aword = ps.stem(aword)
    rootwords.append(aword)
  return rootwords

#Remove uninformative words such as a, an, the
def stopWords(tokenizedtext):
  goodwords = []
  for aword in tokenizedtext:
    if aword not in stopwordsset:
      goodwords.append(aword)
  return goodwords

#Feature reduction: getting the roots of words and graphing only the root words
def lemmatizer(tokenizedtext):
  lemmawords = []
  for aword in tokenizedtext:
    aword = wn.lemmatize(aword)
    lemmawords.append(aword)
  return lemmawords

#Input a list of tokens and return a list of unpunctuated tokens/words
def removePunctuation(tokenizedtext):
  nopunctwords = []
  for aword in tokenizedtext:
    if aword not in punctuation:
      nopunctwords.append(aword)
  cleanedwords = []
  for aword in nopunctwords:
    aword = aword.translate(str.maketrans('', '', string.punctuation))
    cleanedwords.append(aword)
  return cleanedwords


# Identify Unique Words

In [ ]:
from time import sleep 

uniquewords = {}
count = 0

for tweetzipfile in tweetzipfiles:
  zf = zipfile.ZipFile(tweetzipfile)
  for i, obj in enumerate(zf.infolist()):
    tweetjson = json.load(zf.open(obj))
    count += 1
    if count % 1000 == 0:
        print(count)
    
    text = tweetjson['text']
    #Remove emojis
    noemoji = demoji.replace(text, '')
    #Natural language preprocessing: clean the tweet 
    nourlstext = removeURL(noemoji)
    tokenizedtext = tokenize(nourlstext)
    nostopwordstext = stopWords(tokenizedtext)
    lemmatizedtext = lemmatizer(nostopwordstext)
    nopuncttext = removePunctuation(lemmatizedtext)
    
    #print(tokenizedtext)
    #print(nostopwordstext)
    #print(lemmatizedtext)
    #print(nopuncttext)
    #sleep(10)

    for aword in nopuncttext: #Counting number of unique words
        if aword in uniquewords:
            uniquewords[aword] += 1
        if aword not in uniquewords:
            uniquewords[aword] = 1

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000


In [ ]:
#List of unique words: key - word; value = number of times the word appears
#Go through dictionary one at time - if word was mentioned more than n times, include it
wordstoinclude = set()
wordcount = 0
for aword in uniquewords:
    if uniquewords[aword] > 250: #Play with this number to get around 100-200 words to work with
        wordcount += 1
        wordstoinclude.add(aword) #Set of words used as filter - if word is in this set, include it

In [ ]:
print(wordcount)

149


# Create Tweet Edgelist

In [ ]:
edgelist = open('%s/designer15ksemantic.edgelist.for.gephi.csv' % WORKING_DIR,'w')
csvwriter = csv.writer(edgelist)

header = ['Source', 'Target', 'Type']
csvwriter.writerow(header)

print('Writing Edge List')

uniquewords = {}
count = 0

for tweetzipfile in tweetzipfiles:
  zf = zipfile.ZipFile(tweetzipfile)
  for i, obj in enumerate(zf.infolist()):
    tweetjson = json.load(zf.open(obj))
    count += 1
    if count % 1000 == 0:
        print(count)
    
    text = tweetjson['text']
    #Natural language preprocessing: clean the tweet 
    nourlstext = removeURL(text)
    tokenizedtext = tokenize(nourlstext)
    nostopwordstext = stopWords(tokenizedtext)
    lemmatizedtext = lemmatizer(nostopwordstext)
    nopuncttext = removePunctuation(lemmatizedtext)

    goodwords = [] #Only include words mentioned above certain amount of times
    for aword in nopuncttext:
        if aword in wordstoinclude: #Filter if word was in set of words from above
            goodwords.append(aword.replace(',', ''))
            
    allcombos = itertools.combinations(goodwords, 2) #Set this to the number of words wanted in each combination
    for acombo in allcombos:
        row = []
        for anode in acombo:
            row.append(anode)
        row.append('Undirected')
        csvwriter.writerow(row)

edgelist.close()

Writing Edge List
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
